In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

In [ ]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Code above is only for colab, not needed for PC

In [ ]:


import torch.nn as nn
import torch.nn.functional as F

INPUT_DIM = 4
OUTPUT_DIM = 1
class SimplePolicyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(INPUT_DIM, 12)
        self.fc2 = nn.Linear(12, 6)
        self.fc3 = nn.Linear(6, OUTPUT_DIM)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

model = SimplePolicyNet()

In [ ]:
import numpy as np
import torch
def update_weights(model, beta = 0.005):
    new_model = SimplePolicyNet()
    zipped_parameters = zip(model.parameters(), new_model.parameters())
    with torch.no_grad():
        for old_param, new_param in zipped_parameters:
            loc=0#np.random.random()
            scale=1#np.random.random()
            new_param_values = old_param.numpy() + beta * np.random.normal(loc = loc, scale=scale, size=new_param.shape  )
            new_param.data = nn.parameter.Parameter(torch.tensor(new_param_values).float())
    return new_model

In [ ]:
env.action_space.sample()

## CODE BELOW DOES THE SAME THING OF CODE ABOVE
from random import choice
action_space = [1, 0]
choice(action_space)

NameError: ignored

In [ ]:
print(next(model.parameters()))
update_weights(model)
print(next(model.parameters()))
print(next(model.parameters()))


In [ ]:
def evaluate_episode(env, model):

    observation = env.reset()

    max_reward = float("-inf")

    #for i in range(1000):
    total_reward = 0
    while True:

        #env.render()
        
        #your agent goes here
        action = model(torch.tensor(observation).float()).detach().numpy()[0]
        action = int(round(action))
        observation, reward, done, info = env.step(action) 
        total_reward += reward
            
        if done: 
            return total_reward

In [ ]:
from random import choice 
def repopulate_top_N(reward_to_model_dict, N=10, population = 100, beta=0.05):
    top_list = {k: v for k, v in sorted(list(reward_to_model_dict.items()))[-N:]}
    top_list = list(top_list.items())
    new_models = []
    top_performer = top_list[-1][1]
    print("BEST FITNESS TO DATE: {}".format(top_list[-1][0]))
    new_models.append(top_performer)
    while len(new_models) < population:
        fitness, model = choice(top_list)
        new_model = update_weights(model, beta)
        new_models.append(new_model)
    del reward_to_model_dict
    return new_models

def evaluate_new_models(env, new_models ):
    reward_to_model_dict = {}
    for model in new_models:
        fitness = evaluate_episode(env, model)
        reward_to_model_dict[fitness] = model
    del new_models
    return reward_to_model_dict




In [ ]:
from tqdm import tqdm
import gym
# First Iteration
reward_to_model_dict = {0:SimplePolicyNet()}
env = wrap_env(gym.make("CartPole-v0"))

for _ in tqdm(range(100)):
    new_models = repopulate_top_N(reward_to_model_dict, N=3, population = 10, beta = 0.05)
    reward_to_model_dict = evaluate_new_models(env, new_models )



  0%|          | 0/100 [00:00<?, ?it/s]

BEST FITNESS TO DATE: 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  1%|          | 1/100 [00:00<01:36,  1.03it/s]

BEST FITNESS TO DATE: 10.0
BEST FITNESS TO DATE: 10.0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  3%|▎         | 3/100 [00:01<00:34,  2.79it/s]

...

 97%|█████████▋| 97/100 [00:12<00:00,  6.27it/s]

BEST FITNESS TO DATE: 141.0
BEST FITNESS TO DATE: 137.0


 99%|█████████▉| 99/100 [00:12<00:00,  5.92it/s]

BEST FITNESS TO DATE: 188.0
BEST FITNESS TO DATE: 200.0


100%|██████████| 100/100 [00:12<00:00,  7.93it/s]


In [ ]:
fitness, model =  sorted(list(reward_to_model_dict.items()))[-1]


In [ ]:
import gym
import numpy as np
from copy import deepcopy


env = wrap_env(gym.make("CartPole-v0"))

observation = env.reset()

max_reward = float("-inf")

#for i in range(1000):

while True:

    env.render()
    
    #your agent goes here
    action = model(torch.tensor(observation).float()).detach().numpy()[0]
    action = int(round(action))
    observation, reward, done, info = env.step(action) 

    #if max_reward < reward:
    #    reward = max_reward
    #    best_model = deepcopy(model)
        
    if done: 
        break;
            
env.close()
show_video()